In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Lighthouse Labs/LLM-Project/notebooks

Mounted at /content/drive
/content/drive/MyDrive/Lighthouse Labs/LLM-Project/notebooks


In [3]:
# Install necessary libraries
!pip install datasets evaluate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
t

In [4]:
# Load tokenized dataset and fine-tuned model
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [7]:
tokenized_ds = load_from_disk("tokenized_imdb")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
# Load evaluation metrics
import evaluate

accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

In [9]:
# Define compute_metrics function for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy.compute(predictions=predictions, references=labels)
    f1_score = f1.compute(predictions=predictions, references=labels, average='weighted')
    return {**acc, **f1_score}

In [19]:
# Define training arguments for optimization
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results_optimized",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,  # Try changing this to test different learning rates
    num_train_epochs=2,
    evaluation_strategy="epoch",
    save_strategy="no",
    logging_dir="./logs_optimized",
    logging_steps=100,  # Log less frequently
    fp16=True,  # Use mixed precision for faster training
    load_best_model_at_end=False
)

# Use a smaller subset for optimization testing
small_train_ds = tokenized_ds["train"].shuffle(seed=42).select(range(800))
small_eval_ds = tokenized_ds["test"].shuffle(seed=42).select(range(200))


# Create the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_ds,
    eval_dataset=small_eval_ds,
    compute_metrics=compute_metrics
)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [14]:
import os
# Disable wandb and optimize GPU
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [20]:
# Train the model (again on a small subset for tuning purposes)
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.417500,0.374681,0.845000,0.844747


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.417500,0.374681,0.845000,0.844747
2,0.230700,0.501035,0.850000,0.850060


TrainOutput(global_step=400, training_loss=0.3689512491226196, metrics={'train_runtime': 6512.7021, 'train_samples_per_second': 0.246, 'train_steps_per_second': 0.061, 'total_flos': 211947837849600.0, 'train_loss': 0.3689512491226196, 'epoch': 2.0})

In [21]:
# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation Results:", eval_results)

Evaluation Results: {'eval_loss': 0.501035213470459, 'eval_accuracy': 0.85, 'eval_f1': 0.8500600240096038, 'eval_runtime': 204.8296, 'eval_samples_per_second': 0.976, 'eval_steps_per_second': 0.244, 'epoch': 2.0}


In [24]:
label_map = {0: "NEGATIVE", 1: "POSITIVE"}  # Define the mapping based on the model's output



In [22]:
model.save_pretrained("/content/drive/MyDrive/Lighthouse Labs/LLM-Project/notebooks/final_distilbert_model")